In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import os
import glob
import pandas as pd
import pickle
import time
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc,roc_auc_score,precision_recall_curve,average_precision_score,f1_score,precision_score,recall_score
import scipy
warnings.filterwarnings("ignore")
import imageio
from time import sleep
import timeit
import cv2
import shutil
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
physical_cpu_cores = int(multiprocessing.cpu_count()/2)
print(physical_cpu_cores)

In [ ]:
#download audio dataset
if not os.path.exists('./ESC-50-master'):
    if not os.path.exists('./master'):
        !wget https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master
    !unzip master
    if os.path.exists('./master'):
        !rm master

# collect a list of audio files
audio_files = glob.glob('./ESC-50-master/audio/*.wav')

# divide audiofiles into train and test
train_files, test_files = train_test_split(audio_files, test_size=0.2, random_state=42)

# view folder structure and filenames
#!ls -l ESC-50-master


In [ ]:

# view metadata 
#!cat ESC-50-master/meta/esc50.csv

In [ ]:
class audio_search:
    def __init__(self):
        self.audio_files = glob.glob('./ESC-50-master/audio/*.wav')
        self.train_files, self.test_files = train_test_split(audio_files, test_size=0.2, random_state=42)
        self.df = pd.read_csv('ESC-50-master/meta/esc50.csv')
        # find sample rate of files
        self.sr = librosa.get_samplerate(self.audio_files[0])

    # convert audio to spectrogram 
    def to_spectrogram(self, audio_file, win=50, hop_length=100, n_fft=2000, log=True, RGB=False):
        if type(audio_file) == str:
            y, self.sr = librosa.load(audio_file)
            X = S = librosa.feature.melspectrogram(y=y, sr=self.sr, window=win, n_fft=n_fft, hop_length=hop_length)
            if RGB == False:
                if log: S = X = librosa.power_to_db(X, ref=np.max)

            if RGB:

                Y = librosa.power_to_db(X, ref=np.max)
                Z = librosa.feature.melspectrogram(y=y, sr=self.sr, window=200, n_fft=n_fft, hop_length=hop_length)
                S = np.dstack((X, Y, Z))
            return S
        else:
            S = librosa.feature.melspectrogram(y=audio_file, sr=self.sr, hop_length=hop_length, n_fft=n_fft, win_length=win)
            if log: S = librosa.power_to_db(S, ref=np.max)
            return S
    
    def from_spectrogram(self, spectrogram,  hop_length=100, n_fft=2048, win=scipy.signal.hann):
        # undo power_to_db
        #S = librosa.db_to_power(spectrogram, ref=np.max)
        return librosa.feature.inverse.mel_to_audio(spectrogram, sr=self.sr, n_fft=n_fft, hop_length=hop_length, window=win)

    def to_mfcc(self, audio_file):
        y, sr = librosa.load(audio_file)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        return mfccs

    def from_mfcc(self, mfccs):
        return librosa.feature.inverse.mfcc_to_audio(mfccs)

    def to_fourier(self, audio_file):
        y, sr = librosa.load(audio_file)
        fourier = np.fft.fft(y)
        return fourier

    def from_fourier(self, fourier):
        return np.fft.ifft(fourier)

    def spectrogam2d_to_fourier(self, spectrogram):
        return np.fft.fft(spectrogram)

    def spectrogram2d_from_fourier(self, fourier):
        return np.fft.ifft(fourier)

    def to_wavelet(self, audio_file):
        y, sr = librosa.load(audio_file)
        wavelet = scipy.signal.cwt(y, scipy.signal.ricker, np.arange(1, 101))
        return wavelet

    def from_wavelet(self, wavelet):
        return scipy.signal.icwt(wavelet, scipy.signal.ricker, np.arange(1, 101))

    def to_wavelet_2d(self, spectrogram):
        wavelet = scipy.signal.cwt(spectrogram, scipy.signal.ricker, np.arange(1, 101))
        return wavelet

    def from_wavelet_2d(self, wavelet):
        return scipy.signal.icwt(wavelet, scipy.signal.ricker, np.arange(1, 101))

    def play_audio(self, audio_file):
        if type(audio_file) == str:
            return ipd.Audio(audio_file,  rate=self.sr)
        else:
            return ipd.Audio(audio_file, rate=self.sr)

    def visualize_spectrogram(self, audio_file):
        if type(audio_file) == str:
            log_S = self.to_spectrogram(audio_file)
            plt.figure(figsize=(12, 4))
            librosa.display.specshow(log_S, sr=self.sr, x_axis='time', y_axis='mel')
            plt.title('mel power spectrogram ')
            plt.colorbar(format='%+02.0f dB')
            plt.tight_layout()
            plt.show()
        
        else:
            plt.figure(figsize=(12, 4))
            librosa.display.specshow(audio_file, sr=self.sr, x_axis='time', y_axis='mel')
            plt.title('mel power spectrogram ')
            plt.colorbar(format='%+02.0f dB')
            plt.tight_layout()
            plt.show()

    def visualize_mfcc(self, audio_file):
        mfccs = self.to_mfcc(audio_file)
        plt.figure(figsize=(12, 4))
        librosa.display.specshow(mfccs, sr=self.sr, x_axis='time')
        plt.colorbar()
        plt.title('MFCC')
        plt.tight_layout()
        plt.show()

    def visualize_fourier(self, audio_file):
        fourier = self.to_fourier(audio_file)
        plt.figure(figsize=(12, 4))
        plt.plot(fourier)
        plt.title('Fourier')
        plt.show()

    def visualize_wavelet(self, audio_file):
        wavelet = self.to_wavelet(audio_file)
        plt.figure(figsize=(12, 4))
        plt.plot(wavelet)
        plt.title('Wavelet')
        plt.show()

    def visualize_wavelet_2d(self, audio_file):
        wavelet = self.to_wavelet_2d(audio_file)
        plt.figure(figsize=(12, 4))
        plt.plot(wavelet)
        plt.title('Wavelet 2D')
        plt.show()

    def visualize_spectrogram2d(self, audio_file):
        spectrogram = self.to_spectrogram(audio_file)
        plt.figure(figsize=(12, 4))
        plt.plot(spectrogram)
        plt.title('Spectrogram 2D')
        plt.show()

    def visualize_spectrogram2d_fourier(self, audio_file):
        spectrogram = self.to_spectrogram(audio_file)
        fourier = self.spectrogam2d_to_fourier(spectrogram)
        plt.figure(figsize=(12, 4))
        plt.plot(fourier)
        plt.title('Spectrogram 2D Fourier')
        plt.show()

    def visualize_spectrogram2d_wavelet(self, audio_file):
        spectrogram = self.to_spectrogram(audio_file)
        wavelet = self.to_wavelet_2d(spectrogram)
        plt.figure(figsize=(12, 4))
        plt.plot(wavelet)
        plt.title('Spectrogram 2D Wavelet')
        plt.show()

    def visualize_spectrogram2d_mfcc(self, audio_file):
        spectrogram = self.to_spectrogram(audio_file)
        mfcc = self.to_mfcc(spectrogram)
        plt.figure(figsize=(12, 4))
        plt.plot(mfcc)
        plt.title('Spectrogram 2D MFCC')
        plt.show()

    def visualize_spectrogram2d_mfcc_fourier(self, audio_file):
        spectrogram = self.to_spectrogram(audio_file)
        mfcc = self.to_mfcc(spectrogram)
        fourier = self.spectrogam2d_to_fourier(mfcc)
        plt.figure(figsize=(12, 4))
        plt.plot(fourier)
        plt.title('Spectrogram 2D MFCC Fourier')
        plt.show()

    def return_shape_of_audio(self, audio_file):
        y, sr = librosa.load(audio_file)
        return y.shape

    def return_shape_of_spectrogram(self, audio_file):
        log_S = self.to_spectrogram(audio_file)
        return log_S.shape

    def return_shape_of_mfcc(self, audio_file):
        mfccs = self.to_mfcc(audio_file)
        return mfccs.shape

    def return_shape_of_fourier(self, audio_file):
        fourier = self.to_fourier(audio_file)
        return fourier.shape

In [ ]:
# test audio_search class
search = audio_search()
search.visualize_spectrogram('./ESC-50-master/audio/1-137-A-32.wav')


In [ ]:
n_fft, hop_length=2000, 150
win=np.hanning(n_fft)
win = 50
spec = search.to_spectrogram('./ESC-50-master/audio/1-137-A-32.wav', n_fft=n_fft, hop_length=hop_length, win=win, log=False)
aud = search.from_spectrogram(spec, n_fft=n_fft, hop_length=hop_length, win=win)
#aud = aud / np.max(np.abs(aud)) * 32767
#play audio

search.play_audio(aud)

In [ ]:
spec.shape

In [ ]:
#if os.path.exists('./ESC-50-master/mel_spectrograms'):
    #shutil.rmtree('./ESC-50-master/mel_spectrograms')

In [ ]:


if not os.path.exists('./ESC-50-master/mel_spectrograms'):
    os.makedirs('./ESC-50-master/mel_spectrograms')

num_files = len(os.listdir('./ESC-50-master/audio'))
file_names = glob.glob('./ESC-50-master/audio/*.wav')

win=np.hanning(n_fft)

start = timeit.default_timer()

def process_files(wav_path, n_fft, hop_length, win, log, RGB = False, save_format = 'png'):
    img = search.to_spectrogram(wav_path, n_fft=n_fft, hop_length=hop_length, win=win, log=True, RGB=RGB)
    if RGB == False: 
        img = img.reshape(img.shape[0], img.shape[1], 1)
        img = (img - np.min(img)) / (np.max(img) - np.min(img)) * 255
    else:
        img = (img - np.min(img)) / (np.max(img) - np.min(img)) * 255

    
    img_name = wav_path.split('/')[-1].split('.')[0]
    if save_format == 'npz':
        np.savez_compressed('./ESC-50-master/mel_spectrograms/{}.npz'.format(img_name), img)
    else:
        img_path = './ESC-50-master/mel_spectrograms/' + img_name + '.png'
        cv2.imwrite(img_path, img, [cv2.IMWRITE_PNG_COMPRESSION, 9])



In [ ]:
#paralel using joblib

Parallel(n_jobs=physical_cpu_cores)(delayed(process_files)(wav_path, n_fft=n_fft, hop_length=hop_length, win=win, log=True, RGB=False, save_format='png') for wav_path in file_names) # works better on my amd cpu
#Parallel(n_jobs=physical_cpu_cores, backend='multiprocessing')(delayed(process_files)(wav_path, n_fft=n_fft, hop_length=hop_length, win=win, log=True, RGB=False, save_format='png') for wav_path in file_names) #works better on my intel laptop
  
end = timeit.default_timer()
print(end - start)
    